# SCDAA Coursework 2024-2025

## Exercise 1.1 (Solving LQR using from Ricattic ODE)
Write a class which:
1. Can be initialised with the matrices specifying the LQR problem and T > 0 and a time grid (numpy array or torch tensor). 
2. Has a method which will solve (approximate) the associated Ricatti ODE on this time grid. 
3. Has a method that, given one torch 1d-tensor (for time) and another torch 2d-tensor (for space), will return a 1d-torch tensor the control problem value v(t, x) for the given t, x. If t is not on the time grid use tn which belongs to the grid and is the the nearest to t (or the biggest tn smaller than t). 
4. Has a method that, given one torch 1d-tensor (for time) and another torch 2d-tensor (for space), will return a 2d-torch tensor with entries being the optimal Markov control function for the given t, x i.e..

In [23]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.integrate import quad
import torch
from torchdiffeq import odeint

In [25]:
class LQR:
    def __init__(self, H, M, C, D, R, sigma, T, time_grid):
        """
        初始化 LQR 类

        Parameters:
        H, M, C, D, R: 线性二次调节器的矩阵
        sigma: 噪声项
        T: 终止时间
        time_grid: 时间网格 (numpy array)
        """
        self.H = H
        self.M = M
        self.C = C
        self.D = D
        self.R = R
        self.sigma = sigma
        self.T = T
        self.time_grid = time_grid
        self.S_values = self.solve_riccati_ode()

    def riccati_ode(self, t, S_flat):
        """Riccati ODE 求解函数，转换为向量形式"""
        S = S_flat.reshape(2, 2)  # 2x2 矩阵
        S_dot = S @ self.M @ np.linalg.inv(self.D) @ self.M.T @ S - self.H.T @ S - S @ self.H - self.C
        return S_dot.flatten()

    def solve_riccati_ode(self):
        """使用 solve_ivp 求解 Riccati ODE"""
        S_T = self.R.flatten()  # 终止条件 S(T) = R
        sol = solve_ivp(self.riccati_ode, [self.T, 0], S_T, t_eval=self.time_grid[::-1])  # 逆向求解
        S_matrices = sol.y.T[::-1].reshape(-1, 2, 2)  # 转换回矩阵格式
        return dict(zip(self.time_grid, S_matrices))

    def get_nearest_S(self, t):
        """找到最近的 S(t)"""
        nearest_t = self.time_grid[np.argmin(np.abs(self.time_grid - t))]
        return self.S_values[nearest_t]

    def value_function(self, t, x):
        """计算新的 v(t, x) = x^T S(t) x + ∫[t,T] tr(σσ^T S(r)) dr"""
        # 第一部分：x^T S(t) x
        S_t = self.get_nearest_S(t)
        value = np.dot(x.T, S_t @ x)
        
        # 第二部分：积分项 ∫[t,T] tr(σσ^T S(r)) dr
        def integrand(r):
            S_r = self.get_nearest_S(r)
            return np.trace(self.sigma @ self.sigma.T @ S_r)
        
        integral, _ = quad(integrand, t, self.T)  # 使用数值积分计算积分项
        value += integral
        return value

    def optimal_control(self, t, x):
        """计算最优控制 a(t, x) = -D^(-1) M^T S(t) x"""
        S_t = self.get_nearest_S(t)
        return -np.linalg.inv(self.D) @ self.M.T @ S_t @ x
# 示例参数
H = np.array([[0, 1], [-1, 0]])
M = np.array([[0], [1]])
C = np.eye(2)
D = np.array([[1]])
R = np.eye(2)
sigma = np.eye(2) * 0.1
T = 1.0
time_grid = np.linspace(0, T, 100)

# 实例化 LQR 类
lqr = LQR(H, M, C, D, R, sigma, T, time_grid)

# 计算示例
t_test = 0.5
x_test = np.array([1, 0])

print("Value function v(t, x):", lqr.value_function(t_test, x_test))
print("Optimal control a(t, x):", lqr.optimal_control(t_test, x_test))

Value function v(t, x): 1.4788762663397266
Optimal control a(t, x): [-0.10033745]


/tmp/ipykernel_8167/2828252867.py:51: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral, _ = quad(integrand, t, self.T)  # 使用数值积分计算积分项


In [13]:
class LQR:
    def __init__(self, H, M, C, D, R, sigma, T, time_grid, device="cpu"):
        self.device = device
        self.H = H.to(device)
        self.M = M.to(device)
        self.C = C.to(device)
        self.D = D.to(device)
        self.R = R.to(device)
        self.sigma = sigma.to(device)
        self.T = T
        self.time_grid = torch.tensor(time_grid, device=device)
        self.S_values = self.solve_riccati_ode()

    def riccati_ode(self, t, S_flat):
        S = S_flat.view(2, 2)  # 2x2 矩阵
        S_dot = S @ self.M @ torch.linalg.inv(self.D) @ self.M.T @ S - self.H.T @ S - S @ self.H - self.C
        return S_dot.flatten()

    def solve_riccati_ode(self):
        S_T = self.R.flatten().to(self.device)  # 终止条件 S(T) = R
        sol = odeint(self.riccati_ode, S_T, self.time_grid.flip(0))  # 逆向求解
        S_matrices = sol.flip(0).view(-1, 2, 2)  # 转换回矩阵格式
        return dict(zip(self.time_grid.tolist(), S_matrices))

    def get_nearest_S(self, t):
        nearest_t = self.time_grid[torch.argmin(torch.abs(self.time_grid - t))].item()
        return self.S_values[nearest_t]

    def value_function(self, t, x):
        S_t = self.get_nearest_S(t)
        return x.T @ S_t @ x

    def optimal_control(self, t, x):
        S_t = self.get_nearest_S(t)
        return -torch.linalg.inv(self.D) @ self.M.T @ S_t @ x

# 示例参数
device = "cuda" if torch.cuda.is_available() else "cpu"
H = torch.tensor([[0, 1], [-1, 0]], dtype=torch.float32, device=device)
M = torch.tensor([[0], [1]], dtype=torch.float32, device=device)
C = torch.eye(2, device=device)
D = torch.tensor([[1]], dtype=torch.float32, device=device)
R = torch.eye(2, device=device)
sigma = torch.eye(2, device=device) * 0.1
T = 1.0
time_grid = torch.linspace(0, T, 100, device=device)

# 实例化 LQR 类
lqr = LQR(H, M, C, D, R, sigma, T, time_grid, device=device)

# 计算示例
t_test = torch.tensor(0.5, device=device)
x_test = torch.tensor([1, 0], dtype=torch.float32, device=device)

print("Value function v(t, x):", lqr.value_function(t_test, x_test))
print("Optimal control a(t, x):", lqr.optimal_control(t_test, x_test))


Value function v(t, x): tensor(1.4596)
Optimal control a(t, x): tensor([-0.0970])


/tmp/ipykernel_8167/2579258019.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.time_grid = torch.tensor(time_grid, device=device)
/tmp/ipykernel_8167/2579258019.py:31: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  return x.T @ S_t @ x
